In [1]:
import pandas as pd
import numpy as np
import warnings
import time
import csv
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
#import cPickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
#from sklearn.learning_curve import learning_curve

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading the train, test and validation data from the saved** **CSVs**

In [5]:
path = "/content/drive/MyDrive/Data/"
df_train = pd.read_csv(path + "train.csv")
df_test = pd.read_csv(path + "test.csv")
df_valid = pd.read_csv(path + "valid.csv")

In [6]:
df_train

,Unnamed: 0,x_train,y_train
0,0,No plans yet. What are you doing ?,ham
1,1,2marrow only. Wed at &lt;#&gt; to 2 aha.,ham
2,2,Do you like shaking your booty on the dance fl...,ham
3,3,Baaaaabe! I misss youuuuu ! Where are you ? I ...,ham
4,4,Jus ans me lar. U'll noe later.,ham
...,...,...,...
4454,4454,"My planning usually stops at ""find hella weed,...",ham
4455,4455,"Shit that is really shocking and scary, cant i...",ham
4456,4456,Dear how is chechi. Did you talk to her,ham
4457,4457,Watching tv lor... Y she so funny we bluff her...,ham


In [7]:
x_train = df_train["x_train"]
y_train = df_train["y_train"]

In [8]:
x_test = df_test["x_test"]
y_test = df_test["y_test"]

In [9]:
x_valid = df_valid["x_valid"]
y_valid = df_valid["y_valid"]

**Function to split the messages into words**

In [10]:
def split_into_lemmas(message):
    message = str(message).lower()
    words = TextBlob(message).words
    return [word.lemma for word in words]

In [11]:
import nltk

In [12]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

**Naive Bayes Model**

---




In [13]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [14]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7f3f22ad0280>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [15]:
pipeline.score(x_test, y_test)

0.9623655913978495

In [16]:
pipeline.score(x_valid, y_valid)

0.9371633752244165

In [33]:
y_pred_NB = pipeline.predict(x_valid)
pred_data_NB = pd.DataFrame()
pred_data_NB['TEST'] = y_valid
pred_data_NB['PRED_Y'] = y_pred_NB
pred_data_NB

,TEST,PRED_Y
0,ham,ham
1,ham,ham
2,ham,ham
3,ham,ham
4,ham,ham
...,...,...
552,ham,ham
553,ham,ham
554,ham,ham
555,ham,ham


In [34]:
confusion_matrix(y_valid, y_pred_NB)

array([[483,   0],
       [ 35,  39]])

In [35]:
print(classification_report(y_valid, y_pred_NB))

              precision    recall  f1-score   support

         ham       0.93      1.00      0.97       483
        spam       1.00      0.53      0.69        74

    accuracy                           0.94       557
   macro avg       0.97      0.76      0.83       557
weighted avg       0.94      0.94      0.93       557



**Decision Tree Classifier**

---



In [17]:
from sklearn.tree import DecisionTreeClassifier 

In [18]:
pipeline_decisicionTree = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', DecisionTreeClassifier(random_state=42,max_depth = 10)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [19]:
pipeline_decisicionTree.fit(x_train,y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7f3f22ad0280>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 DecisionTreeClassifier(max_depth=10, random_state=42))])

In [20]:
pipeline_decisicionTree.score(x_test, y_test)

0.956989247311828

In [21]:
pipeline_decisicionTree.score(x_valid, y_valid)

0.9515260323159784

In [22]:
y_pred_tree = pipeline_decisicionTree.predict(x_valid)
pred_data = pd.DataFrame()
pred_data['TEST'] = y_valid
pred_data['PRED_Y'] = y_pred_tree
pred_data

,TEST,PRED_Y
0,ham,ham
1,ham,ham
2,ham,ham
3,ham,ham
4,ham,ham
...,...,...
552,ham,ham
553,ham,ham
554,ham,ham
555,ham,ham


In [23]:
confusion_matrix(y_valid, y_pred_tree)

array([[479,   4],
       [ 23,  51]])

In [24]:
print(classification_report(y_valid, y_pred_tree))

              precision    recall  f1-score   support

         ham       0.95      0.99      0.97       483
        spam       0.93      0.69      0.79        74

    accuracy                           0.95       557
   macro avg       0.94      0.84      0.88       557
weighted avg       0.95      0.95      0.95       557



**Random Forest Classifier**

---



In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
pipeline_randomForest = Pipeline([
    ('bow', CountVectorizer(analyzer=split_into_lemmas)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', RandomForestClassifier(n_estimators = 100,random_state=42,max_depth = 10)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [27]:
pipeline_randomForest.fit(x_train,y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7f3f22ad0280>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 RandomForestClassifier(max_depth=10, random_state=42))])

In [28]:
pipeline_randomForest.score(x_test, y_test)

0.9121863799283154

In [29]:
pipeline_randomForest.score(x_valid, y_valid)

0.8886894075403949

In [30]:
y_pred_forest = pipeline_randomForest.predict(x_valid)
pred_data = pd.DataFrame()
pred_data['TEST'] = y_valid
pred_data['PRED_Y'] = y_pred_forest
pred_data

,TEST,PRED_Y
0,ham,ham
1,ham,ham
2,ham,ham
3,ham,ham
4,ham,ham
...,...,...
552,ham,ham
553,ham,ham
554,ham,ham
555,ham,ham


In [31]:
confusion_matrix(y_valid, y_pred_forest)

array([[483,   0],
       [ 62,  12]])

In [32]:
print(classification_report(y_valid, y_pred_forest))

              precision    recall  f1-score   support

         ham       0.89      1.00      0.94       483
        spam       1.00      0.16      0.28        74

    accuracy                           0.89       557
   macro avg       0.94      0.58      0.61       557
weighted avg       0.90      0.89      0.85       557



**Although the accuracy for the Random Forest is less as compared to the other two models. But the precision is better for Random Forest. So, in this case it is prefereble to choose Random Forest over other models.**